In [6]:
import pandas as pd
from decouple import config, AutoConfig
from sqlalchemy import create_engine

In [9]:
config = AutoConfig(search_path='./')

user = config('user')
password = config('password')
host = config('host')
port = config('port')
db = config('db')

connection = f"postgresql://{user}:{password}@{host}:{port}/{db}"

engine = create_engine(connection)

### Question 1. Knowing docker tags

Which tag has the following text? - Write the image ID to the file

    • --idfile string 

### Question 2. Understanding docker first run

How many python packages/modules are installed?

    • 3  

### Question 3. Count records

How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18.

Remember that lpep_pickup_datetime and lpep_dropoff_datetime columns are in the format timestamp (date and hour+min+sec) and not in date.

In [10]:
query = """
    SELECT
        COUNT(*)
    FROM 
        green_taxi_trips
    WHERE 
        CAST(lpep_pickup_datetime AS DATE) = '2019-09-18'
        AND CAST(lpep_dropoff_datetime AS DATE) = '2019-09-18';
"""

pd.read_sql(query, con=engine)

,count
0,15612


### Question 4. Largest trip for each day

Which was the pick up day with the largest trip distance Use the pick up time for your calculations.

    • 2019-09-26 

In [11]:
query = """
    SELECT
        a.lpep_pickup_datetime
        ,a.trip_distance
        --,b.rn
    FROM 
        green_taxi_trips a
    JOIN (
        SELECT
            lpep_pickup_datetime
            ,ROW_NUMBER() OVER(PARTITION BY CAST(lpep_pickup_datetime AS DATE) ORDER BY trip_distance DESC) as rn
        FROM 
            green_taxi_trips
        ) as b on b.lpep_pickup_datetime = a.lpep_pickup_datetime
    WHERE 
        b.rn = 1
    ORDER BY 
        trip_distance DESC
    LIMIT 1;
"""

pd.read_sql(query, con=engine)

,lpep_pickup_datetime,trip_distance
0,2019-09-26 19:32:52,341.64


### Question 5. The number of passengers

Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?

    • "Brooklyn" "Manhattan" "Queens" 

In [12]:
query = """
    SELECT 
        "Borough",
        SUM(total_amount) AS sum_total_amount
    FROM
        green_taxi_trips AS a
    JOIN zones ON "LocationID" = "PULocationID"
    WHERE
        CAST(lpep_pickup_datetime AS DATE) = '2019-09-18'
        AND "Borough" != 'Unknown'
    GROUP BY
        "Borough"
    HAVING SUM(total_amount) > 50000;
"""

pd.read_sql(query, con=engine)

,Borough,sum_total_amount
0,Brooklyn,96333.24
1,Manhattan,92271.30
2,Queens,78671.71


### Question 6. Largest tip

For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip? We want the name of the zone, not the id.

    • JFK Airport 

In [13]:
query = """
    SELECT 
        "Zone",
        "tip_amount",
        "DOLocationID",
        "PULocationID"
    FROM green_taxi_trips AS a
    JOIN zones AS z1 ON "LocationID" = "DOLocationID"
    WHERE CAST(lpep_pickup_datetime AS DATE) between '2019-09-01' and '2019-09-30'
    AND "PULocationID" IN (
        SELECT "LocationID"
        FROM zones
        WHERE "Zone" = 'Astoria'
        )
    ORDER BY tip_amount DESC
    LIMIT 1;
"""

pd.read_sql(query, con=engine)

,Zone,tip_amount,DOLocationID,PULocationID
0,JFK Airport,62.31,132,7
